## GEOCODER for lgbtqspaces


In [1]:
# imports and other needed things
# https://api.mapbox.com/{endpoint}?access_token={your_access_token}

import geocoder
import pandas as pd
import numpy as np
import os
import time

# If use mapbox, then an access token is required - obtained through creating an account in Mapbox. 
access_token = ""

GEOCODER Documentation/Guide: https://geocoder.readthedocs.io/

Current notes:
    This implementation is adding every csv file into the code and running all the addresses (so it takes a long time to run in generael).    
    Can take just the geocoder and experiment with single addresses first before doing every file! 

- change `files` to corresponding directory
- change `location` to corresponding `city, state` format to ensure geocoding accuracy
- please ensure that corresponding directory does not have same file name as output files already (otherwise code will take 2x the time to run)
- current implementation is to use mapbox, can be adjusted with what Geocoder's service applies (check documentation)

In [19]:
# Add data files - have change depend on csv and excel locations
directory = "seattle_m/"
location = "Seattle, WA"

# Find all files in the set directory above 
files = os.listdir(directory)
df_wa_men = []

# Read into directory and put all files into list
for i in range(len(files)):
    temp_df = pd.read_csv(directory + files[i])
    temp_df.columns = temp_df.columns.str.lower()
    df_wa_men.append(temp_df)

# *Only specifically for Seattle Data (remove all rows with no data in all columns)
# Can comment out for data not pertaining to Seattle_m
df_wa_men[1] = df_wa_men[1][0:21]

# NULL values in address currently changed to name
for i in range(len(df_wa_men)):
    for j in range(len(df_wa_men[i].address)):
        # make each address field into a valid string
        df_wa_men[i].address[j] = str(df_wa_men[i].address[j])
        if "NULL" in df_wa_men[i].address[j]:
            df_wa_men[i].address[j] = df_wa_men[i].name[j]
        elif "NaN" in df_wa_men[i].address[j]:
            df_wa_men[i].address[j] = df_wa_men[i].name[j]
len(df_wa_men)

C:\Users\kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


7

In [22]:
## Geocoding for lat and lng
## ** Code is also timed to make comparisons
t0 = time.time()
for i in range(len(df_wa_men)):
    lat = []
    long = []
    for j in range(len(df_wa_men[i].address)): 
        g = geocoder.mapbox(df_wa_men[i].address[j] + " " + location, key=access_token)
        # if mapbox code throws error 404, then currently replace with Seattle lat long
        if "404" in g.status:
            lat.append(47.6038)
            long.append(-122.3301)
            # Print out i,j // i = file, j = row observation
            print(i, j)
        else:
        # Otherwise this is data obtained from the geocoder API
            lat.append(g.latlng[0])
            long.append(g.latlng[1])
            
    df_wa_men[i]["Latitude"] = lat
    df_wa_men[i]["Longitude"] = long
t1 = time.time()
print(t1-t0)


Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/115 1/2 1st Ave S Seattle, WA.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/115%201/2%201st%20Ave%20S%20Seattle,%20WA.json?access_token=pk.eyJ1Ijoia2V2aW5rb2NodW55dSIsImEiOiJja3BkdDRkMzYxaHJiMnBvMWNlZ21iZm12In0.EgOe8AAJuApJrrEDtc62IQ


3 38


Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Archidocesan Catholic Gay/Lesbian Ministry Seattle, WA.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Archidocesan%20Catholic%20Gay/Lesbian%20Ministry%20Seattle,%20WA.json?access_token=pk.eyJ1Ijoia2V2aW5rb2NodW55dSIsImEiOiJja3BkdDRkMzYxaHJiMnBvMWNlZ21iZm12In0.EgOe8AAJuApJrrEDtc62IQ


4 65
39.16551446914673


In [23]:
# output file with {name of file}_geocoded.csv
for i in range(len(files)):
    df_wa_men[i].to_csv(directory + files[i][:-4] + "_geocoded.csv", index=True)

### Experiments

In [3]:
import requests
## add API key here to use google's API!
api_key= ''

base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
address = 'GSBA Seattle, WA'
params = {
    'key': api_key,
    'address': address
}

response = requests.get(base_url, params=params)
response.json()

{'results': [{'address_components': [{'long_name': '322',
     'short_name': '322',
     'types': ['subpremise']},
    {'long_name': '400', 'short_name': '400', 'types': ['street_number']},
    {'long_name': 'East Pine Street',
     'short_name': 'E Pine St',
     'types': ['route']},
    {'long_name': 'Capitol Hill',
     'short_name': 'Capitol Hill',
     'types': ['neighborhood', 'political']},
    {'long_name': 'Seattle',
     'short_name': 'Seattle',
     'types': ['locality', 'political']},
    {'long_name': 'King County',
     'short_name': 'King County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Washington',
     'short_name': 'WA',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '98122', 'short_name': '98122', 'types': ['postal_code']}],
   'formatted_address': '400 E Pine St #322, Seattle, WA 98122, USA',
   